In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix
from nltk.stem.snowball import SnowballStemmer

from nltk import word_tokenize, WordNetLemmatizer, PorterStemmer
from nltk import pos_tag
from nltk import ngrams
from nltk import sent_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
array_col = ['skincare', 'hair', 'make-up', 'other']

In [3]:
df_train = pd.read_csv("data/hackathon_loreal_train_set.csv")
df_train

,Unnamed: 0,text,skincare,hair,make-up,other,index
0,0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0
1,1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1
2,2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2
3,3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3
4,4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...
10567,13209,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209
10568,13210,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210
10569,13212,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212
10570,13213,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213


# watch data

In [4]:
df_train.columns

Index(['Unnamed: 0', 'text', 'skincare', 'hair', 'make-up', 'other', 'index'], dtype='object')

# delete one columns

In [5]:
df_train = df_train.drop(['Unnamed: 0'], axis = 1)
df_train

,text,skincare,hair,make-up,other,index
0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0
1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1
2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2
3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3
4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...
10567,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209
10568,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210
10569,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212
10570,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213


## see disparite

In [6]:
for val in array_col:
    len_result = len(df_train[df_train[val] == 1])
    print(val, " : " ,len_result)

skincare  :  1303
hair  :  1272
make-up  :  1907
other  :  6667


# preprocessing one text

In [7]:
Onetext = df_train['text'].iloc[2]

In [8]:
import re

# Function to remove emoji.
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
                           
    return emoji_pattern.sub(r'', string)

In [9]:
emoji(Onetext)

'Just in case no one has told you today YOU ARE STRONG BEAUTIFUL I LOVE YOU  Por si no te lo han dicho hoy TU ERES FUERTE BONITA Y TE AMO  Wicked Gel Liner coupon code BIRDY Hydrating spray code BIRDY pearl radiance primer foundation in almond shape tape contour concealer hoola contour stick bronzer powder translucent powder rose gold highlight code BIRDY whirl lip liner Nude potion BIRDY'

In [10]:
OnetextLower = Onetext.lower()

In [11]:
tokenizeSingletext = word_tokenize(OnetextLower)

In [12]:
lemmatizer = WordNetLemmatizer()
lemmatizeSingleText = list(map(lemmatizer.lemmatize, tokenizeSingletext))

In [13]:
stop_punctuation = [':', '(', ')', '/', '|', ',', ']', ';',
                    '.', '*', '#', '"', '&', '~', '``',
                    '-', '_', '\\', '@','?','!','\'', '[', '<', '>', "’"]
TextCleanPonctuation = [word for word in lemmatizeSingleText if word not in stop_punctuation]

In [14]:
stoplist = stopwords.words('english')
TextCleanPonctuationWord = [word for word in TextCleanPonctuation if word not in stoplist]

In [15]:
from textblob import TextBlob
regroupe_var = ' '.join(TextCleanPonctuationWord)

result = TextBlob(regroupe_var)
result.tags

[('case', 'NN'),
 ('one', 'CD'),
 ('ha', 'NN'),
 ('told', 'VBD'),
 ('today', 'NN'),
 ('strong', 'JJ'),
 ('beautiful', 'JJ'),
 ('love', 'NN'),
 ('🌹', 'NNP'),
 ('por', 'NN'),
 ('si', 'NN'),
 ('te', 'NN'),
 ('lo', 'NN'),
 ('han', 'NN'),
 ('dicho', 'NN'),
 ('hoy', 'NN'),
 ('tu', 'NN'),
 ('eres', 'VBZ'),
 ('fuerte', 'JJ'),
 ('bonita', 'NN'),
 ('te', 'NN'),
 ('amo', 'NN'),
 ('🌹', 'NNP'),
 ('wicked', 'VBD'),
 ('gel', 'JJ'),
 ('liner', 'NN'),
 ('coupon', 'NN'),
 ('code', 'NN'),
 ('birdy', 'IN'),
 ('hydrating', 'VBG'),
 ('spray', 'NN'),
 ('code', 'NN'),
 ('birdy', 'NN'),
 ('pearl', 'NN'),
 ('radiance', 'NN'),
 ('primer', 'NN'),
 ('foundation', 'NN'),
 ('almond', 'NN'),
 ('shape', 'NN'),
 ('tape', 'NN'),
 ('contour', 'VBP'),
 ('concealer', 'NN'),
 ('hoola', 'NN'),
 ('contour', 'NN'),
 ('stick', 'NN'),
 ('bronzer', 'NN'),
 ('powder', 'NN'),
 ('translucent', 'NN'),
 ('powder', 'NN'),
 ('rose', 'VBD'),
 ('gold', 'JJ'),
 ('highlight', 'NN'),
 ('code', 'NN'),
 ('birdy', 'IN'),
 ('whirl', 'NN'),
 ('li

# functions

In [23]:
import re

with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text


# Function to remove emoji.
def del_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
                           
    return emoji_pattern.sub(r'', string)




def text_clean_ponctuation(text):
    stop_punctuation = [':', '(', ')', '/', '|', ',', ']', ';',
                    '.', '*', '#', '"', '&', '~', '``',
                    '-', '_', '\\', '@','?','!','\'', '[', '<', '>', '£', '$', '”', "\u2063", "•", "'s", "“"]
    stoplist = stopwords.words('english')

    TextCleanPonctuation = [word for word in text if word not in stop_punctuation and word not in stoplist]
    return TextCleanPonctuation

def regroupe_text(list_word):
    return ' '.join(list_word)

def preprocessing(singleText):
    lemmatizer = WordNetLemmatizer()
    singleText = del_emoji(singleText)
    LowerSingleText = singleText.lower()
    tokenizeSingletext = word_tokenize(LowerSingleText)
    lemmatizeSingleText = list(map(lemmatizer.lemmatize, tokenizeSingletext))
    TextCleanPonctuation = text_clean_ponctuation(lemmatizeSingleText)

    return TextCleanPonctuation

def clean_text(text):
    text_preprocessed = preprocessing(text)
    return regroupe_text(text_preprocessed)

In [24]:
result = clean_text(df_train["text"].iloc[0])
result

"skeeterbytes wrote bassam guy wrote n't anything longer 60mm except 99 plastic fantastic '' 40-150. 've never really tried avian photography would like get decent shot unlike opportunity arises used c-af+tr retrospect since turkey lawyer refused fly despite plea would s-af done better mf peaking magnify exposed lawyer dressed dark suit bright cloudy sky meteorologist say smoke remnant ca northern va wa +2 +2.5 ev much encouraged chroma 'm great judging distance would say wa 35+ meter away pretty intense purple fringing lightroom ha effective one-click remover 'm sure software ha hard control know panasonic 7-14 4 never use c-af+tr preferring standard c-af s-af instance s-af fine attorney n't going anywhere pay bill get s-af+magnify allow tweak focus plus want stop lens bit sharpen except 7-14 nothing else f2.8 5.6 seems stopped 'll watch tried magnify wa shaky long fl 'll try lower scale factor also raise shutter speed freeze flapping 're iso200 good lens within realm 40-150 pro leagu

# On applique cela pour toute la base de donnée

In [25]:
df_train["clean_content"] = df_train.text.apply(clean_text)
df_train["clean_content"]

0        skeeterbytes wrote bassam guy wrote n't anythi...
1        refueling long week ’ meant drink prefer match...
2        case one ha told today strong beautiful love p...
3        forget get sunshine safely head link bio see s...
4        abstract background human dramatically changed...
                               ...                        
10567    day 273 outifit thor gold foil back bling eter...
10568    order study various trend pattern prevailing c...
10569    brand ha yet registered influenster work brand...
10570    regime get rid acne scar get way smoother heal...
10571    true used work caddie trump national nj donny ...
Name: clean_content, Length: 10572, dtype: object

In [26]:
from collections import Counter

def number_of_word_each_class(df):
    array_col = ['skincare', 'hair', 'make-up', 'other']

    for row_class in array_col:
        df_one_class = df[df[row_class] == 1]

        top = Counter([item for sublist in df_one_class["clean_content"] for item in sublist])
        temp = pd.DataFrame(top.most_common(20))
        temp.columns = ['Common_words','count']
        temp.style.background_gradient(cmap='Blues')

def tokenization(text):
    return word_tokenize(text)

def see_word_frequency_class(df):
    df["clean_content"] = df.clean_content.apply(tokenization)
    number_of_word_each_class(df)
    df["clean_content"] = df.clean_content.apply(regroupe_text)

see_word_frequency_class(df_train)
df_train

,text,skincare,hair,make-up,other,index,clean_content
0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0,skeeterbytes wrote bassam guy wrote n't anythi...
1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1,refueling long week ’ meant drink prefer match...
2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2,case one ha told today strong beautiful love p...
3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3,forget get sunshine safely head link bio see s...
4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4,abstract background human dramatically changed...
...,...,...,...,...,...,...,...
10567,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209,day 273 outifit thor gold foil back bling eter...
10568,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210,order study various trend pattern prevailing c...
10569,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212,brand ha yet registered influenster work brand...
10570,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213,regime get rid acne scar get way smoother heal...


# Model

In [28]:
from sklearn import preprocessing

df_train_clean = df_train.drop(["text"], axis = 1)
X = df_train['clean_content']
Y = df_train.drop(['clean_content', 'index'], axis = 1)
X

0        skeeterbytes wrote bassam guy wrote n't anythi...
1        refueling long week ’ meant drink prefer match...
2        case one ha told today strong beautiful love p...
3        forget get sunshine safely head link bio see s...
4        abstract background human dramatically changed...
                               ...                        
10567    day 273 outifit thor gold foil back bling eter...
10568    order study various trend pattern prevailing c...
10569    brand ha yet registered influenster work brand...
10570    regime get rid acne scar get way smoother heal...
10571    true used work caddie trump national nj donny ...
Name: clean_content, Length: 10572, dtype: object

In [29]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [222]:
tfidf = TfidfTransformer()

In [223]:
tfidf.fit(x_train)

ValueError: could not convert string to float: '“ ’ mad bonkers completely head ’ tell secret best people are . lewis carrol alice wonderland first diy spa day home ’ quite go um planned day get aeryn eat 🥒 smile face good day first definitely last casa winderley spa “ mom daughter experience wa inspired skinrenewalsa proudly brought u tammytaylornailssouthafrica mrs_south_africa skincare sponsor lamellersa haircare sponsor marcanthony_sa ... really missed nail sponsor tammy taylor ever tried give wriggly giggly five-year old mani/pedi led trying get bright purple “ frozen nail polish outta white linen 🤦\u200d'

In [ ]:
x_vectorized_train = tfidf.transform(x_train)
x_vectorized_test = tfidf.transform(x_test)

In [ ]:
model.fit(x_vectorized_train, y_train)

In [ ]:
y_predicted = model.predict(x_vectorized_test)

In [221]:
pipe = make_pipeline(TfidfTransformer())
pipe.fit(x_train)
X_feat_train = pipe.transform(x_train)
X_feat_test = pipe.transform(x_test)

X_feat_train_numpy = np.asarray(X_feat_train)

ValueError: could not convert string to float: '“ ’ mad bonkers completely head ’ tell secret best people are . lewis carrol alice wonderland first diy spa day home ’ quite go um planned day get aeryn eat 🥒 smile face good day first definitely last casa winderley spa “ mom daughter experience wa inspired skinrenewalsa proudly brought u tammytaylornailssouthafrica mrs_south_africa skincare sponsor lamellersa haircare sponsor marcanthony_sa ... really missed nail sponsor tammy taylor ever tried give wriggly giggly five-year old mani/pedi led trying get bright purple “ frozen nail polish outta white linen 🤦\u200d'

# Model

In [208]:
np.shape(x_train)
type(x_train)

scipy.sparse.csr.csr_matrix

In [219]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(128, input_shape=(8457,), activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['F1 score'])

model.fit(X_feat_train_numpy, y_train, epochs=5)

#model.evaluate(x_test, y_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_matrix).

In [195]:
#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(n_estimators=50)
#clf.fit(feat_train, y_train)

TypeError: '<' not supported between instances of 'float' and 'str'

In [182]:
#feat_test = pipe.transform(x_test)
#clf.score(feat_test, y_test)

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

# visualisation result

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
score = clf.predict_proba(feat_test)
score
#fpr, tpr, th = roc_curve(y_list_test, score[:, 1])

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(4,4))
ax.plot([0, 1], [0, 1], 'k--')
aucf = auc(fpr, tpr)
ax.plot(fpr, tpr, label='auc=%1.5f' % aucf)
ax.set_title('Courbe ROC - classifieur de sentiments')
ax.legend();